<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/gretelai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install datasets -q

In [9]:
# Mount Google Drive (for Colab)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from datasets import load_dataset

# Convert dataset to OAI messages
system_message = """You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}"""

def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message.format(schema=sample["context"])},
      {"role": "user", "content": sample["question"]},
      {"role": "assistant", "content": sample["answer"]}
    ]
  }


def create_conversation0(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message.format(schema=sample["sql_context"])},
      {"role": "user", "content": sample["sql_prompt"]},
      {"role": "assistant", "content": sample["sql"]}
    ]
  }


# Load dataset from the hub
dataset = load_dataset("b-mc2/sql-create-context", split="train")
gretel_text_to_sql = load_dataset("gretelai/synthetic_text_to_sql", split="train")

#dataset = dataset.shuffle().select(range(12500))
dataset = dataset.select(range(12500))
gretel_text_to_sql  = gretel_text_to_sql.select(range(12500))

# Convert dataset to OAI messages
dataset = dataset.map(create_conversation, remove_columns=dataset.features,batched=False)
gretel_text_to_sql = gretel_text_to_sql.map(create_conversation0,remove_columns=gretel_text_to_sql.features,batched=False)

# split dataset into 10,000 training samples and 2,500 test samples
dataset = dataset.train_test_split(test_size=2500/12500)
gretel_text_to_sql = gretel_text_to_sql.train_test_split(test_size=2500/12500)

#%cd /content/
# save datasets to disk
dataset["train"].to_json("/content/drive/MyDrive/datasets/train_dataset.json", orient="records")
dataset["test"].to_json("/content/drive/MyDrive/datasets/test_dataset.json", orient="records")

# save datasets to disk
gretel_text_to_sql["train"].to_json("/content/drive/MyDrive/datasets/gretelai_train_dataset.json", orient="records")
gretel_text_to_sql["test"].to_json("/content/drive/MyDrive/datasets/gretelai_test_dataset.json", orient="records")


Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

1879570

In [23]:
gretel_text_to_sql

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 2500
    })
})

In [26]:
DATASET_FILE='/content/drive/MyDrive/datasets/gretelai_test_dataset.json'

In [27]:
gretelai_eval_dataset = load_dataset("json", data_files=DATASET_FILE, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [28]:
gretelai_eval_dataset

Dataset({
    features: ['messages'],
    num_rows: 2500
})

In [29]:
gretelai_eval_dataset[0]

{'messages': [{'content': "You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE open_data_initiatives (id INT, sector VARCHAR(20), status VARCHAR(10)); INSERT INTO open_data_initiatives (id, sector, status) VALUES (1, 'justice', 'open'), (2, 'transportation', 'open'), (3, 'education', 'closed'), (4, 'education', 'open');",
   'role': 'system'},
  {'content': 'What is the percentage of successful open data initiatives in the education sector?',
   'role': 'user'},
  {'content': "SELECT 100.0 * COUNT(CASE WHEN status = 'open' THEN 1 END) / COUNT(*) FROM open_data_initiatives WHERE sector = 'education';",
   'role': 'assistant'}]}

In [34]:
gretelai_eval_dataset[0]['messages'][0]['content'][153:]

"CREATE TABLE open_data_initiatives (id INT, sector VARCHAR(20), status VARCHAR(10)); INSERT INTO open_data_initiatives (id, sector, status) VALUES (1, 'justice', 'open'), (2, 'transportation', 'open'), (3, 'education', 'closed'), (4, 'education', 'open');"

In [35]:
gretelai_eval_dataset[0]['messages'][1]['content']

'What is the percentage of successful open data initiatives in the education sector?'

In [36]:
gretelai_eval_dataset[0]['messages'][2]['content']

"SELECT 100.0 * COUNT(CASE WHEN status = 'open' THEN 1 END) / COUNT(*) FROM open_data_initiatives WHERE sector = 'education';"